In [1]:
#Import Libraries

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import torch
import torchvision
import albumentations as A

np.random.seed(3)

In [2]:
#Import Dataset and impute values

df_train = pd.read_csv("C:\\Users\Abhishek Ramesh\\Google Drive\\1USF\\Spring 2021\\Math373\\HW\\Final\\training.csv", header=0, sep=',', quotechar='"')
df_test = pd.read_csv("C:\\Users\Abhishek Ramesh\\Google Drive\\1USF\\Spring 2021\\Math373\\HW\\Final\\test.csv")

whichfacepoints = pd.read_csv("C:\\Users\Abhishek Ramesh\\Google Drive\\1USF\\Spring 2021\\Math373\\HW\\Final\\IdLookupTable.csv", header=0, sep=',', quotechar='"')

df_train = df_train.fillna(0) #Replace null values with 0

In [3]:
#Class for CNN model

class FacialKeyPoints_CNN(torch.utils.data.Dataset):

  def __init__(self, df_x, df_y):
    self.df_x = df_x
    self.df_y = df_y


  def __len__(self):
    return len(self.df_x)

  def __getitem__(self, idx):
    row_x = self.df_x.iloc[idx]

    row_y = self.df_y.iloc[idx]
    
    x = np.float32(row_x.split( )) / 255
    x = np.reshape(x, (1,96,96)) #Model requires a reshape of 96x96 pixels

    
    y = np.float32(row_y.to_numpy())

    return x,y

In [4]:
#Methods for Face Dataset Class

class FaceDataset(torch.nn.Module):
    def __init__(self):

        super().__init__()

        self.conv1 = torch.nn.Conv2d(1, 32, 3)
        self.conv2 = torch.nn.Conv2d(32, 64, 3)

        self.dense1 = torch.nn.Linear(30976, 30)    

        self.pool = torch.nn.MaxPool2d(2)

        self.flatten = torch.nn.Flatten()

        self.ReLU = torch.nn.ReLU()

    def forward(self, x):
        #Convolution 1
        x = self.conv1(x)
        x = self.ReLU(x)
        x = self.pool(x)

        #Convolution 2
        x = self.conv2(x)
        x = self.ReLU(x)
        x = self.pool(x)

        #Output
        x = self.flatten(x)
        x = self.dense1(x) 

        return x

In [5]:
# Create batches for CNN Model

def createBatch(x_train, y_train, x_val, y_val):
  dataset_train = FacialKeyPoints_CNN(x_train, y_train)
  dataset_val = FacialKeyPoints_CNN(x_val, y_val)

  dataloader_train = torch.utils.data.DataLoader(dataset_train, batch_size = 64, shuffle = True)
  dataloader_val = torch.utils.data.DataLoader(dataset_val, batch_size = 64, shuffle = True)

  return dataset_train, dataset_val, dataloader_train, dataloader_val

In [6]:
#Training CNN model

def trainCNN(dataset_train, dataset_val, dataloader_train, dataloader_val):
  model = FaceDataset()
  device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
  model = model.to(device)

  loss_fun = torch.nn.MSELoss() #Using Mean-Squared Error Loss since it's a regression problem
  optimizer = torch.optim.RMSprop(model.parameters(), lr = 0.001)

  num_epochs = 5
  N_train = len(dataset_train)
  N_val = len(dataset_val)

  train_losses = []
  val_losses = []

  for ep in range(num_epochs):

    model.train()
    train_loss = 0.0
    batch_idx = 0

    for X_batch, Y_batch in dataloader_train:

      X_batch = X_batch.to(device)
      Y_batch = Y_batch.to(device)

      N_batch = X_batch.shape[0]

      outputs = model(X_batch)
      loss_oneBatch = loss_fun(outputs, Y_batch)

      model.zero_grad()
      loss_oneBatch.backward()
      optimizer.step()


      train_loss += loss_oneBatch * N_batch

    model.eval()
    val_loss = 0.0
    for X_batch, Y_batch in dataloader_val:

      X_batch = X_batch.to(device)
      Y_batch = Y_batch.to(device)

      with torch.no_grad():
        N_batch = X_batch.shape[0]
        outputs = model(X_batch)
        loss_oneBatch = loss_fun(outputs, Y_batch)
        val_loss += loss_oneBatch * N_batch

    train_losses.append(train_loss / N_train)
    val_losses.append(val_loss / N_val)

    print('epoch: ', ep, ' train loss: ', (train_loss), ' val loss: ', (val_loss))


  return model, train_losses, val_losses

In [7]:
#Plot objective function vs epoch

def plot_objectivefunction(train_losses, val_losses):
  plt.plot(train_losses, label = 'Training loss')
  plt.plot(val_losses, label = 'Val loss')
  plt.legend(loc = 'upper right')
  plt.title('Objective Function Value vs. Epoch')

In [ ]:
# Main Function
X = df_train.iloc[:,30]
y = df_train.iloc[:, :30]


x_train, x_val, y_train, y_val = train_test_split(X, y, test_size=0.2)

print('CNN Model')
dataset_train, dataset_val, dataloader_train, dataloader_val = createBatch(x_train, y_train, x_val, y_val)
model, train_losses, val_losses = trainCNN(dataset_train, dataset_val, dataloader_train, dataloader_val)

CNN Model
epoch:  0  train loss:  tensor(2583791.2500, device='cuda:0', grad_fn=<AddBackward0>)  val loss:  tensor(592233.3125, device='cuda:0')


In [ ]:
#Plot Objective Function value

plot_objectivefunction(train_losses, val_losses)